# Nextclade Dataset Generator 

Hello there! In this report, I will describe the process I followed to generate a custom Avian Influenza Nextclade dataset using sequence data available online. 

## Objectives

This investigation and resulting workflow was inspired by both the lack of an existing collection of Influenza H5 clade-defining mutations and an accepted H5NX Nextclade dataset, which depends upon the former. In short, this process sought to first, reverse engineer the clade-defining mutations of Influenza H5NX by analyzing large collections of annotated sequence data online and, using these definitions, build a dataset that enables Nextclade to effectively call H5 clades. 

Another software being used to call H5 clades is LABEL. We opted to spend the time to engineer this Nextclade dataset as opposed to using LABEL for a few reasons: 

1. LABEL's source code is completely hidden. It is offered to users as an executable download without visible source code on a repository. Therefore, we are unable to make modifications or investigate unexpected results. 
2. Related to the above point, LABEL is currently unable to call subclades of the highly relevant 2.3.4.4 clade (2.3.4.4b, 2.3.4.4c, etc). As calling these subclades is one of our primary objectives, LABEL in its current state does not meet our requirements. 
3. Our workflows already utilize Nextclade for Influenza H1 and H3 clade-calling. Maintaining consistency between outputs would be preferred over managing two different output schemas.  


## Files Needed

A finalized Nextclade dataset requires the following 7 files. The amount of work needed to re-engineer each of these files is as follows:

| File                   | Effort           |
|:-----------------------|:-----------------|
| genemap.gff            | minimal          |
| primers.csv            | none (identical) | 
| qc.json                | none (identical) | 
| reference.fasta        | minimal          | 
| tag.json               | minimal          |
| tree.json              | **extensive**    | 
| virus_properties.json  | none (identical) | 


In truth, about 99% of the work in this workflow goes into generating this new `tree.json` file, which is more specifically a Nextstrain tree generated using Nextstrain's `augur` bioinformatics pipeline. 
This tree file contains multiple layers of annotations (nucleotide/amino acid substitutions, ancestral nodes, clade labels) arranged in JSON format such that it can easily be served as a dashboard app using `auspice`. 
Nextclade software uses this tree as a reference point to know what clade labels to assign to incoming query sequences, which is why tuning it correctly is the most important piece of this workflow. 
Everything from this point forward is dedicated towards building this `tree.json` file so that it captures the correct clade label information. 



## Data Collection

The ability to call clades using Nextclade directly depends on the annotated sequences we use to build this Nextstrain tree. 
Therefore, this tree must have a healthy collection of sequences across all clades of interest. 
So, I began compiling a comprehensive collection of Avian Influenza Hemagluttinin (HA) sequences, importantly, with corresponding clade labels, from three main sources: 

1. Dr. Louise Moncla's [Avian Flu Nextstrain Build](https://github.com/nextstrain/avian-flu), in tandem with NCBI Genbank
2. GISAID Database
3. BV-BRC Database



### 1. Dr. Moncla's Flu Build

Dr. Louise Moncla and colleagues have impressively compiled the names and corresponding clade labels of 19,691 H5NX Influenza isolates in [the following TSV file](https://github.com/nextstrain/avian-flu/blob/master/clade-labeling/h5nx-clades.tsv) of their Avian Flu Nextstrain build. I saw this as an excellent starting point off of which I could build a large database of clade data. Since Dr. Moncla's annotations only had sequence names available, this source required some additional scripting steps to generate the final collection of FASTA sequences and corresponding clade labels. 

#### Sequence Retrieval 

I used a Python script to query the GenBank API with Biopython's Entrez submodule to grab the corresponding FASTA sequences (shown below). It first searches the sequence name with "HA" to target hemagluttanin, then retrieves the accession number of the top search result and grabs its FASTA sequence. 

In [1]:
from Bio import Entrez

# "names" variable contains the first column of the h5nx-clades.tsv file linked above

accnos = []
outfasta = open('h5nx.fasta', 'w')

for n, accno in enumerate(names):
	# SEARCH STEP 1: search for top accession number hits based on the sequence name 
	handle = Entrez.esearch(db="nucleotide", retmax=5, term=f"({accno}) AND (HA)", idtype="acc")
	record = Entrez.read(handle)
	handle.close()
	if int(record['Count']) > 0:
		accnos += record['IdList']

		# SEARCH STEP 2: fetches the FASTA sequence given the accession number of the top hit
		query = Entrez.efetch(db='nucleotide', id=record['IdList'][0], rettype='fasta', retmode='text')

		for i in query:
			outfasta.write(i)

outfasta.close()

NameError: name 'names' is not defined

#### Sequence Header Cleaning
The resulting FASTA entries from GenBank all tend to have the following general format:   `>CY028940.1 Influenza A virus (A/chicken/Shantou/28/2002(H5N1)) segment 4 sequence` . To tidy these up, I parse and reformat these headers into a more succinct and useful format using regular expressions in the script "filter_rename.py". The new headers have the following format:  `>CY028940.1|A/chicken/Shantou/28/2002(H5N1)|A|H5N1`. 

#### Metadata Generation
The `parse` function under the `augur` bioinformatics toolkit is an incredibly useful command for parsing FASTA sequence headers. With the above FASTA headers in a cleaner format, I then ran `augur parse` on them to generate a cleaned FASTA file and a metadata file. 

```
augur parse --sequences h5nx.fasta --output-sequences h5nx-parsed.fasta --output-metadata h5nx-metadata.tsv --fields name strain type subtype 
```
The FASTA file now has cleaned headers and looks like this: 

```
>AB621347
TTCACTCTGTCAAAATGGAGAAAATAGTGCTTCTCTTTACAACAATCAGCCTTGTTAAAA
GCGATCATATTTGCATTGGTTATCATGCAAATAACTCGACAGAGCAGGTTGACACAATAA
...
>JF732739
ATGGAGAAAATAGTACTTCTTCTTGCAATAGTCAGTCTTGTTAAAAGTGATCAGATTTGC
ATTGGTTACCATGCAAACAACTCGACAGAGCAGGTTGACACAATAATGGAAAAGAACGTT
...
```


The metadata coming from `augur parse` looks like this (below), however, you will notice that clade information is missing. 

```
name		strain			type	subtype	
CY040926        A/duck/Laos/P0161/2007  A       H5N1
CY040910        A/duck/Laos/P0127/2007  A       H5N1 
CY041014        A/chicken/Laos/33/2008  A       H5N1
```

To add in the clade data, I then used the `merge_clade.py` script (found in the bin folder) to merge this column into the metadata, resulting in the final common metadata schema I use for all further analyses:

```
name		strain			type	subtype	clade
CY040926        A/duck/Laos/P0161/2007  A       H5N1    2.3.4
CY040910        A/duck/Laos/P0127/2007  A       H5N1    2.3.4
CY041014        A/chicken/Laos/33/2008  A       H5N1    2.3.4
```



### 2. GISAID Database

Retrieving sequence data from GISAID involved a simple, but manual process. This process involves the folloing steps:

1. Login and navigate to the EpiFlu search page of GISAID, filtering the search specifically for influenza A H5 segments.
2. Use the filtering criteria to select a single clade of interest, and search for it. 
3. Select all sequences and use the "Download" button, loading a pop-up window 
4. Check that the specified heading matches your accepted schema. Then select the "HA" segment only and download the file 
5. Rename the file to the name of the clade. 
6. Repeat steps 2-5 for all H5 clades of interest. 



## Nextstrain Build 

Once we have a cleaned FASTA file and a metadata file with clade information, we can run this through the `augur` pipeline and produce a candidate `tree.json` file. 
I have two scripts in the `bin/` folder dedicated to running the `augur` workflow: 

1. run_augur.sh
2. run_augur2.sh 

### run_augur.sh

The first script encapsulates the resource-heavy steps of the pipeline: 1) alignment and 2) tree building. . It can be run in the following manner 

```
bash run_augur.sh [SEQUENCE_FASTA] [NUM_THREADS]
```


### run_augur2.sh

The second script handles all the metadata-oriented tree annotation steps of the pipeline that follow the initial tree construction step.  It can be run as follows:

```
bash run_augur2.sh [SEQUENCE_METADATA]
```

**NOTE:**
- both scripts will create output directories in the location it is run
- both scripts have the H5 hemagluttinin reference name hard-coded into the script as `NC_007362`
- both scripts expect the `assets/h5_ref.fasta` and `assets/h5_ref.gff` files to be in the same directory


## Generating Clade Definitions

You may notice that the above Augur pipeline stops prior to running `augur clades`, the step where all sequences are assigned clade definitions. 
**This is because the critical file that outlines all clade-defining mutations for `augur clades` (the `clades.tsv` file) does not exist for Avian Influenza (at least to our knowledge).
This key file must therefore be created by reverse engineering the mutations found in all our sequences.**

### Clade Prediction Pipeline

The largest piece of this workflow is the `extract_mutations.py` script, responsible for the following steps:

1. Performing a nucleotide pairwise alignment and amino acid pairwise alignment between each input sequence and the H5 HA reference: `NC_007362`. 

2. Extracting all nucleotide and AA mutations from these alignments.

3. Grouping sequences by their source-defined clade definitions, and extracting the most prevalent mutations within a given clade (its clade-defining mutations). For this step, the cutoff of a "clade-defining mutation" can differ greatly between clades. While I start looking for mutations that are present >95% of the clade population, almost limitless tuning can be done here for each of the 50+ clades. For instance, some clades may be called more effectively using the mutations found in >80% of sequences, while other clades work better with the mutations present in >99%. 

4. Generating a custom `clades.tsv` file in a format accepted by `augur clades`. 

5. Using this new `clades.tsv` file to run `augur clades`. This run is indicative of how Nextclade will perform at clade label prediction.

6. Comparing the `augur` clade predictions with the source-defined clade definitions, thereby getting a sense of accuracy / sensitivity / specificity. 

### Tuning Process

To improve the performance of `augur clades`, I iterate over Steps 3 - 6 in this workflow several 10s, perhaps 100s of times. In each iteration, I find the clades that `augur` is struggling to identify and adjust the defining mutations in these clades accordingly. 

#### Thresholding 
Many times, it is possible to see better clade predictions by tuning the "prevalence threshold", which is the prevalence a mutation needs to be present at in the clade population to be considered "clade-defining". Tuning these thresholds for each of the 50+ clades turns into a sensitivity / specificity trade-off. The logic here is as follows: 

- Lowering the "prevalence threshold" will introduce more mutations to the clade definition. It is more lenient at letting mutations into the definition. However, in turn, this makes the clade definition more specific and precise, since there are more mutations that must be present in order to call the clade. 
- Raising the "prevalence threshold" will reduce the mutation of mutations in the clade definition. It is more stringent at what mutations are allowed into the definition. However, in turn, this makes the clade definition more sensitive, since there are fewer mutations needed to make the clade call. 


#### Last Resort -- Node Flipping

Some cases of node definitions are simply not possible to resolve using the threshold approach above. 
This is often caused by cases where the clade labels provided by online sources are not supported by their genetic content. 
These mislabeling errors can prevent the above algorithm from arriving at a clean set of defining mutations that perfectly captures all members of the clade. 

This brute force approach involves the following steps:

1. Generating a best estimate tree.json file after running both `augur clades` and `augur export v2`. Viewing the tree as an app using `auspice view`. 

2. Manually searching sequences whose clade is struggling to be captured properly, and identifying the ID of the single tree node whose subtree best captures the vast majority of true clade sequences. 

3. Using code extracted from `augur` source repository, extract the mutations defined at the key basal tree node identified above. Then overwrite this clade's mutation list with this mutation set extracted from the tree. 

Some of the clades that encountered trouble converging and required this "node flipping" technique were: 

- 2.3.4.4
- 2.3.4.4d
- 2.3.4.4e
- 2.3.4.4g
- 2.1.3.2
- 2.1.3.2a
- 2.1.3.2b
- 2.2.2.1